In [1]:
# imports
from inspect import signature
import control
import numpy
from matplotlib import pyplot as plt
import scipy
import timeit
from ipywidgets import interact, interactive, interact_manual
from IPython.display import display
import IPython.display as dis
import ipywidgets as wid

In [2]:
# inputs
def step(t, start, stop):
    if t<start:
        return 0
    else:
        return 1

def rect(t, start, drop, stop):
    if t>=start and t<stop:
        return 1
    else:
        return 0

def doublet(t, start, drop, rise, stop):
    if t>=start and t<drop:
        return 1
    elif t>=drop and t<rise:
        return -1
    else:
        return 0
    
input_dict = {'Step': step, 
              'Rect': rect, 
              'Doublet': doublet}

In [3]:
# read data
def read(file):
    df = pandas.read_csv(file)
    ydata = df.y.values - df.y[0]
    udata = df.u.values - df.u[0]
    tdata = df.t.values
    return [ydata, udata, tdata]

# save data
def save_data(data, save_name):
    df = pandas.DataFrame(data, columns=['y','u','t'])
    df.to_csv(save_name)
    return None

In [4]:
#  noise

def uniform_noise(data, m): #currently does not include 1
    ydata, udata, t = data
    noise = [random.uniform(-1,1)*m for i in range(len(ydata))]
    noisy = ydata + noise
    return noisy

def pseudo_noise(data, m): #currently only returns int values
    ydata, udata, t = data
    noise = [random.randrange(start=-1,stop=1)*m for i in range(len(ydata))]
    noisy = ydata + noise
    return noisy


In [5]:
# system

def FO(K, τ):
    return control.tf(K, [τ, 1])

def create_system(Numerator, Denominator):
    return control.tf(Numerator, Denominator)

system_dict = {'First Order': FO,
              'Create System': create_system}

In [119]:
# Techniques

# def ARMAX(data):
#     ydata, udata, t = data
#     start = timeit.default_timer()
#     ARMA_results = sm.tsa.ARMA(ydata, order=(2,1), exog = udata).fit()
#     y_armax = ARMA_results.predict(0, len(t)-1, udata) + ydata[0]
#     end = timeit.default_timer()
#     runtime = end - start
#     return y_armax, runtime

# def ARX(data):
#     ydata, udata, t = data
#     start = timeit.default_timer()
#     ARX_results = sm.tsa.AR(ydata).fit()
#     y_arx = ARX_results.predict()
#     end = timeit.default_timer()
#     runtime = end - start
#     t_arx = time[:len(y_arx)]
#     return y_arx, runtime

def DE(function, Bounds):
    return scipy.optimize.differential_evolution(function, Bounds)

def LSTSQ(func, Initial_Guess): #ig = initial guess 
    return scipy.optimize.least_squares(func, Initial_Guess).x
    
tech_dict = {'Differential Evolution': DE,
            'LSTSQ': LSTSQ}

In [120]:
# Misc

def err(params, ts, ym, us):
    num = [params[0]]
    den = params[1:]
    est_sys = control.tf(num, den)
    tsim, ysim, xsim = control.forced_response(est_sys, T=ts, U=us)
    return sum((ym - ysim)**2)

def res(params, ts, ym, us):
    num = [params[0]]
    den = params[1:]
    est_sys = control.tf(num, den)
    tsim, ysim, xsim = control.forced_response(est_sys, T=ts, U=us)
    plt.plot(tsim, ysim, label="Estimation")
    return None

In [121]:
def sim(input='step', system='Create System', technique='Differential Evolution'):

    u = input_dict[input]
    u_sig = u.__code__.co_varnames[1:]
    c_input_params = {}
    c_input_wid = {}
    for var in u_sig:
        c_input_wid[var] = wid.IntSlider(value=1, min=0, max=600, description=var, disabled=False)
        display(c_input_wid[var])
    
    sys = system_dict[system]
    sys_sig = sys.__code__.co_varnames
    c_sys_params = {}
    c_sys_wid = {}
    for var in sys_sig:
        c_sys_wid[var] = wid.Text(value='1;1', description=var, disabled=False)
        display(c_sys_wid[var])
        
    tech = tech_dict[technique]
    tech_sig = tech.__code__.co_varnames[1:]
    for names in tech_sig:
        display(names+":")
    c_tech_params = {}
    c_tech_wid = {}
    for var1 in tech_sig:
        for var2 in sys_sig:
            var = str(var1) + str(var2)
            c_tech_wid[var] = wid.Text(value='0.1,10;0.1,10', description=var2, disabled=False)
            display(c_tech_wid[var])
    
    run_sim = wid.Button(description="Run")
    output = wid.Output()
    
    display(run_sim, output)

    def run_sim_clicked(b):
        
        with output:
            dis.clear_output(True)
            
            for var in u_sig:
                c_input_params[var] = c_input_wid[var].value
            
            for var in sys_sig:
                c_sys_params[var] = [float(i) for i in c_sys_wid[var].value.split(';')]
            
            ts = numpy.linspace(0, c_input_params['stop'])
            us = [u(t, *c_input_params.values()) for t in ts]
            system = create_system(*c_sys_params.values()) 
            tm, ym, xm = control.forced_response(system, T=ts, U=us)
            plt.plot(tm, ym, '--', label="Real")
            
            bounds = []
            
            for var in c_tech_wid.keys():
                c_tech_params[var] = [[float(j) for j in [i.split(',') for i in c_tech_wid[var].value.split(';')][k]] for k in range(len(c_tech_wid[var].value.split(';')))]
                for vals in c_tech_params[var]:
                    bounds.append(vals)

            def err_wrap(params):
                return err(params, ts, ym, us)
        
            def res_wrap(params):
                return res(params, ts, ym, us)
        
            st = timeit.default_timer()
            coeff = tech(err_wrap, bounds).x

            et = timeit.default_timer()
            print("Parameters estimated as {} in {} seconds".format(coeff/coeff[-1], et-st))
            print("Time taken: {}".format(et-st))
            res_wrap(coeff)
            plt.legend()            
            plt.show()
    
    run_sim.on_click(run_sim_clicked)

In [122]:
interact(sim, 
         input=[keys for keys,dict in input_dict.items()], 
         system=[keys for keys,dict in system_dict.items()], 
         technique=[keys for keys,dict in tech_dict.items()])
# First Order system option is redundant - might remove the choice as it can easily be typed in

interactive(children=(Dropdown(description='input', options=('Step', 'Rect', 'Doublet'), value='Step'), Dropdo…

<function __main__.sim(input='step', system='Create System', technique='Differential Evolution')>